In [ ]:
import pandas as pd

In [ ]:
df = pd.read_pickle(
    "/home/thomas/repos/tarification-solidaire-strasbourg/server/files/resultats_cts_2023-10-04T17-59-53_thomas.xlsx_base.pickle"
)

In [ ]:
df_hors_handicap = df[~df.titre_fichier.str.contains("PMR")]

In [ ]:
df.groupby("sample_id").individu_id.count()

In [ ]:
df_hors_handicap.groupby("sample_id").individu_id.count()

In [ ]:
df_hors_handicap.columns

In [ ]:
px = (
    df_hors_handicap[df_hors_handicap.sample_id == 0]
    .groupby("pu_calc")
    .ajustement_mensuel_num.count()
    .cumsum()
)

In [ ]:
px

In [ ]:
v = []
for n, gdf in df_hors_handicap.groupby("sample_id"):
    qfs = gdf.qf_fiscal.sort_values()
    v.append(
        pd.DataFrame(
            {
                "qf": qfs.iloc[px.iloc[0 : (len(px) - 1)]].values,
                "seuil": [*range(len(px) - 1)],
            }
        )
    )
seuils = pd.concat(v)
seuils.groupby("seuil").qf.median()

In [ ]:
pxa = (
    df_hors_handicap[df_hors_handicap.sample_id == 0]
    .groupby("pu_calc")
    .ajustement_mensuel_num.sum()
    .cumsum()
)

In [ ]:
pxa

In [ ]:
v = []
for n, gdf in df_hors_handicap.groupby("sample_id"):
    qfs = gdf.qf_fiscal.sort_values()
    v.append(
        pd.DataFrame(
            {
                "qf": qfs.iloc[pxa.iloc[0 : (len(pxa) - 1)]].values,
                "seuil": [*range(len(pxa) - 1)],
            }
        )
    )
seuils = pd.concat(v)
seuils.groupby("seuil").qf.median()

In [ ]:
v = []
for n, gdf in df_hors_handicap.groupby("sample_id"):
    sgdf = gdf.sort_values("qf_fiscal")
    qfs = sgdf.qf_fiscal
    adj = sgdf.ajustement_mensuel_num.cumsum()
    qq = [qfs[adj.gt(pxa[v])].iloc[0] for v in pxa.index[0 : (len(pxa) - 1)]]
    v.append(
        pd.DataFrame(
            {
                "qf": qq,
                "seuil": [*range(len(pxa) - 1)],
            }
        )
    )
seuils = pd.concat(v)
seuils.groupby("seuil").qf.median()

In [ ]:
v = []
for n, gdf in df.groupby("sample_id"):
    sgdf = gdf.sort_values("qf_fiscal")
    qfs = sgdf.qf_fiscal
    adj = sgdf.ajustement_mensuel_num.cumsum()
    qq = [qfs[adj.gt(pxa[v])].iloc[0] for v in pxa.index[0 : (len(pxa) - 1)]]
    v.append(
        pd.DataFrame(
            {
                "qf": qq,
                "seuil": [*range(len(pxa) - 1)],
            }
        )
    )
seuils = pd.concat(v)
seuils.groupby("seuil").qf.median()

In [ ]:
base_df = pd.read_pickle(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/mobilite/5-10/resultats_cts_2023-10-05T00-16-05_thomas.xlsx_base.pickle"
)

In [ ]:
cst_df = pd.read_pickle(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/mobilite/5-10/resultats_cts_2023-10-05T00-16-05_thomas.xlsx_seuils_equilibre_recettes.pickle"
)

In [ ]:
cst_df_yc_hand = pd.read_pickle(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/mobilite/5-10/resultats_cts_2023-10-05T00-28-40_thomas.xlsx_!seuils_equilibre_recettes_yc_handicap.pickle"
)

In [ ]:
nb_df = pd.DataFrame(
    {
        "base": base_df[~base_df.titre_fichier.str.contains("PMR")]
        .groupby(["sample_id", "pu_calc"])
        .ajustement_mensuel_num.sum()
        .groupby("pu_calc")
        .median(),
        "copil": base_df[~base_df.titre_fichier.str.contains("PMR")]
        .groupby(["sample_id", "pu_calc_r"])
        .ajustement_mensuel_num.sum()
        .groupby("pu_calc_r")
        .median(),
        "cst": cst_df[~cst_df.titre_fichier.str.contains("PMR")]
        .groupby(["sample_id", "pu_calc_r"])
        .ajustement_mensuel_num.sum()
        .groupby("pu_calc_r")
        .median(),
        "cst_h": cst_df_yc_hand[~cst_df.titre_fichier.str.contains("PMR")]
        .groupby(["sample_id", "pu_calc_r"])
        .ajustement_mensuel_num.sum()
        .groupby("pu_calc_r")
        .median(),
    }
)
nb_df

In [ ]:
nb_df.plot.bar()

In [ ]:
nb_df.to_excel("nb_par_prix.xlsx")

In [ ]:
comm_df = pd.read_pickle(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/mobilite/5-10/resultats_cts_2023-10-05T00-45-24_thomas.xlsx_combinaison_lin_exp.pickle"
)

In [ ]:
comm_df.groupby(["sample_id", "pu_calc_r"]).ajustement_mensuel_num.sum().groupby(
    "pu_calc_r"
).median()

In [ ]:
import numpy as np

In [ ]:
determine_prix_avant = lambda x: pd.Series.mode(x).min()
move_df = pd.DataFrame(
    {
        "prix_avant": comm_df.groupby("individu_id").pu_calc.agg(determine_prix_avant),
        "prix après": comm_df.groupby("individu_id").pu_calc_r.median(),
    }
)

In [ ]:
c = move_df.prix_avant.round(2).astype("category")
move_df["prix avant"] = c.cat.rename_categories(
    [round(r, 2) for r in c.dtype.categories.values]
)

In [ ]:
move_df["différence"] = move_df["prix après"] - move_df["prix_avant"]
move_df["ratio"] = move_df["prix après"] / move_df["prix_avant"]

In [ ]:
move_df

In [ ]:
df_tp = move_df[move_df.prix_avant > 26]

In [ ]:
dd = df_tp.groupby(["prix_avant", "prix après"]).ratio.count()
dd

In [ ]:
dd.name = "count"

In [ ]:
totaux = dd.groupby("prix_avant").sum()
totaux

In [ ]:
cible = dd * 1 / 2

In [ ]:
base = cible.groupby("prix_avant").sum()
base

In [ ]:
comp = totaux - base

In [ ]:
cible.index[0][0]

In [ ]:
cible.index[-1][0]

In [ ]:
cible[(cible.index[0][0], cible.index[-1][0])] += comp.iloc[0]

In [ ]:
cible[(cible.index[-1][0], cible.index[-1][0])] += comp.iloc[1]

In [ ]:
assert (cible.groupby("prix_avant").sum() == totaux).all()

In [ ]:
fcible = cible.reset_index()

In [ ]:
(fcible["prix après"] * fcible["count"]).sum()

In [ ]:
df_tp["prix après"].sum()

In [ ]:
(fcible["prix après"] * fcible["count"]).sum() - df_tp["prix après"].sum()

In [ ]:
cible[(cible.index.levels[0].max(), cible.index.levels[1].max())]

In [ ]:
import seaborn

In [ ]:
gp = seaborn.ecdfplot(move_df, stat="count", x="prix après", hue="prix avant")
gp.set(xticks=range(0, 52, 5))
gp.set(title="Répartition des abonnements aux prix actuels vers des nouveaux tarifs")

In [ ]:
gd = seaborn.ecdfplot(move_df, x="différence", hue="prix avant", stat="count")
gd.axvline(0, c="black", zorder=-1, linestyle="--", linewidth=0.5)
gd.set(title="Différence entre les nouveaux prix et les prix actuels")

In [ ]:
gr = seaborn.ecdfplot(move_df, x="ratio", hue="prix avant", stat="count")
gr.axvline(1, c="black", zorder=-1, linestyle="--", linewidth=0.5)
gr.set(title="Rapport des nouveaux prix sur les prix actuels")

In [ ]:
gp.figure.savefig("1.evol_prix.png", dpi=300)
gd.figure.savefig("2.evol_diff.png", dpi=300)
gr.figure.savefig("3.evol_ratio.png", dpi=300)

In [ ]:
oc_df = pd.read_csv(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/mobilite/2023-10-04_CTS_QFMES.csv",
    delimiter=";",
    encoding="windows-1250",
    index_col=0,
    decimal=",",
    usecols=[
        "ID",
        "ColNameK",
        "COMP_FAM",
        "DERN_LIB_PRODUIT",
        "Ville",
        "QFEMS",
        "NSERIE",
    ],
)

In [ ]:
oc_df.sort_index()

In [ ]:
s1 = oc_df[oc_df.NSERIE == 1]

In [ ]:
s1.groupby("Ville").COMP_FAM.count().sort_values(ascending=False)[0:5]

In [ ]:
viv = s1[
    s1.Ville.isin(
        s1.groupby("Ville").COMP_FAM.count().sort_values(ascending=False)[0:10].index
    )
]

In [ ]:
viv.groupby("Ville").QFEMS.describe()

In [ ]:
seaborn.ecdfplot(viv, x="QFEMS", hue="Ville")

In [ ]:
viv.plot.hist(
    "QFEMS",
    alpha=0.5,
)

In [ ]:
mm = oc_df.groupby("ID").QFEMS.median()

In [ ]:
st = pd.DataFrame(
    {
        "qf": mm,
        "titre": oc_df.groupby("ID").DERN_LIB_PRODUIT.first(),
        "titre": oc_df.groupby("ID").DERN_LIB_PRODUIT.first(),
        "titre": oc_df.groupby("ID").DERN_LIB_PRODUIT.first(),
    }
)

In [ ]:
st

In [ ]:
mm.plot.hist(bins=50)

In [ ]:
seaborn.ecdfplot(mm)

In [ ]:
import sysa

sys.path.append("../technique")
from utils import *

In [ ]:
from scenario import build_reform

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
import ezodf

In [ ]:
input_file = "/home/thomas/repos/tarification-solidaire-strasbourg/server/files/baremes_cts_2023-10-04T23-23-54_thomas.ods"
doc = ezodf.opendoc(input_file)

In [ ]:
reform = build_reform(base, doc.sheets["!seuils_equilibre_recettes"])

In [ ]:
reform.parameters.metropoles.strasbourg.tarification_solidaire.bareme_emeraude

In [ ]:
rdf[rdf.pu_calc_r == 0]